In [25]:
from pynq import Overlay, MMIO
from time import sleep
from datetime import datetime

class NEC_IR_Controller:
    def __init__(self, bitfile_path,
                 tx0_base=0x41210000,  # AXI GPIO for Transmitter 0
                 tx1_base=0x41220000,  # AXI GPIO for Transmitter 1
                 btn_base=0x41200000,   # AXI GPIO for shared Button
                 command_dict = {
                     '-':0x07,
                     '+':0x15,
                     '0':0x16,
                     '1':0x0C,
                     '2':0x18,  
                     '3':0x5E,
                     '4':0x08,
                     '5':0x1C,
                     '6':0x5A,
                     '7':0x42,
                     '8':0x52,
                     '9':0x4A,
                 }):
        
        print(f"[DEBUG] Loading overlay from: {bitfile_path}")
        self.ol = Overlay(bitfile_path)
        self.ol.download()
        print("[DEBUG] Overlay loaded and downloaded.")

        print("[DEBUG] Initializing MMIO handles...")
        self.tx0 = MMIO(tx0_base, 0x10000)
        self.tx1 = MMIO(tx1_base, 0x10000)
        self.btn = MMIO(btn_base, 0x10000)
        print(f"[DEBUG] MMIO handles initialized: tx0_base={hex(tx0_base)}, tx1_base={hex(tx1_base)}, btn_base={hex(btn_base)}")

        self.command_dict = command_dict

        print("[DEBUG] Configuring GPIO directions for transmitters and button...")
        # Configure transmitters as outputs
        self.tx0.write(0x04, 0x00000000)  # ch1 as output
        self.tx0.write(0x0C, 0x00000000)  # ch2 as output
        self.tx1.write(0x04, 0x00000000)  # ch1 as output
        self.tx1.write(0x0C, 0x00000000)  # ch2 as output
        
        # Button should be output to pulse it
        self.btn.write(0x04, 0x00000000)  # output
        print("[DEBUG] GPIO directions configured.")

        self.NEC_HOLD_TIME = 0.3       # 300 ms hold for NEC frame
        self.BTN_PULSE_WIDTH = 0.001   # 1 ms button pulse

        print("[DEBUG] Clearing transmitter and button states at init...")
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        self._btn_low()
        print("[DEBUG] Initialization complete.")

    def _clear_tx(self, tx_ip):
        print(f"[DEBUG] Clearing transmitter lines")
        tx_ip.write(0x00, 0x00000000)  # CH1 low
        tx_ip.write(0x08, 0x00000000)  # CH2 low

    def _btn_high(self):
        print("[DEBUG] Button HIGH pulse.")
        self.btn.write(0x00, 0x1)

    def _btn_low(self):
        print("[DEBUG] Button LOW.")
        self.btn.write(0x00, 0x0)

    def send_commands(self, tx0_addr, tx0_cmd, tx1_addr, tx1_cmd):
        print(f"[DEBUG] Sending commands: tx0_addr={tx0_addr}, tx0_cmd={tx0_cmd}, tx1_addr={tx1_addr}, tx1_cmd={tx1_cmd}")
        
        # Clear everything first
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        self._btn_low()
        sleep(self.BTN_PULSE_WIDTH)

        # Set up the commands
        self.tx0.write(0x00, tx0_addr & 0xFF)
        self.tx0.write(0x08, tx0_cmd & 0xFF)
        print(f"[DEBUG] TX0 written: addr={hex(tx0_addr & 0xFF)}, cmd={hex(tx0_cmd & 0xFF)}")

        self.tx1.write(0x00, tx1_addr & 0xFF)
        self.tx1.write(0x08, tx1_cmd & 0xFF)
        print(f"[DEBUG] TX1 written: addr={hex(tx1_addr & 0xFF)}, cmd={hex(tx1_cmd & 0xFF)}")

        # Pulse the button to trigger
        self._btn_high()
        sleep(self.BTN_PULSE_WIDTH)
        self._btn_low()

        # Hold for NEC frame duration
        sleep(self.NEC_HOLD_TIME)
        
        # Clear everything
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        print("[DEBUG] Commands sent and lines cleared.")

    def cleanup(self):
        print("[DEBUG] Cleaning up: clearing transmitters and button.")
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        self._btn_low()
        print("[DEBUG] Cleanup complete.")

    def assign_values(self, tx0_addr, tx0_cmd_char, tx1_addr, tx1_cmd_char):
        print(f"[DEBUG] Assigning values: tx0_addr={tx0_addr}, tx0_cmd='{tx0_cmd_char}', tx1_addr={tx1_addr}, tx1_cmd='{tx1_cmd_char}'")
        
        try:
            hex_cmd_0 = self.command_dict[tx0_cmd_char]
            hex_cmd_1 = self.command_dict[tx1_cmd_char]
        except KeyError as e:
            print(f"[ERROR] KeyError: '{e.args[0]}' not found in command_dict")
            raise

        print(f"[DEBUG] Command chars mapped to hex: '{tx0_cmd_char}'={hex(hex_cmd_0)}, '{tx1_cmd_char}'={hex(hex_cmd_1)}")

        self.tx0.write(0x00, tx0_addr & 0xFF)
        self.tx0.write(0x08, hex_cmd_0 & 0xFF)
        print(f"[DEBUG] TX0 assigned: addr={hex(tx0_addr & 0xFF)}, cmd={hex(hex_cmd_0 & 0xFF)}")

        self.tx1.write(0x00, tx1_addr & 0xFF)
        self.tx1.write(0x08, hex_cmd_1 & 0xFF)
        print(f"[DEBUG] TX1 assigned: addr={hex(tx1_addr & 0xFF)}, cmd={hex(hex_cmd_1 & 0xFF)}")
    def reverse_bits_8bit(self, value):
        
    # Ensure we're working with 8 bits only
        value = value & 0xFF
    
    # Reverse the bits
        reversed_val = 0
        for i in range(8):
            if (value >> i) & 1:
                reversed_val |= 1 << (7 - i)
    
        return reversed_val
 
    def send_command_full(self, tx0_addr, tx1_addr, cmd0_str, cmd1_str):
        """Send sequences of commands to both transmitters"""
        print(f"[DEBUG] send_command_full: tx0_addr={tx0_addr}, tx1_addr={tx1_addr}, cmd0='{cmd0_str}', cmd1='{cmd1_str}'")
        
        # Check if strings are same length
        if len(cmd0_str) != len(cmd1_str):
            print(f"[WARNING] Command strings have different lengths: {len(cmd0_str)} vs {len(cmd1_str)}")
        
        # Get the minimum length to iterate safely
        min_len = min(len(cmd0_str), len(cmd1_str))
        
        for idx in range(min_len):
            cmd0_char = cmd0_str[idx]
            cmd1_char = cmd1_str[idx]
            
            print(f"[DEBUG] Loop {idx}: command chars: tx0='{cmd0_char}', tx1='{cmd1_char}'")
            
            # Clear before each iteration
            self._clear_tx(self.tx0)
            self._clear_tx(self.tx1)
            self._btn_low()
            sleep(self.BTN_PULSE_WIDTH)
            
            # Assign values
            self.assign_values(tx0_addr, cmd0_char, tx1_addr, cmd1_char)
            
            # Pulse button
            print(f"[DEBUG] Pulsing button for commands '{cmd0_char}' and '{cmd1_char}'...")
            self._btn_high()
            sleep(self.BTN_PULSE_WIDTH)
            self._btn_low()
            
            # Hold for NEC frame
            sleep(self.NEC_HOLD_TIME)
        
        print("[DEBUG] send_command_full finished.")
    def assign_values_reversed(self, tx0_addr, tx0_cmd_char, tx1_addr, tx1_cmd_char):
        print(f"[DEBUG] Assigning values: tx0_addr={tx0_addr}, tx0_cmd='{tx0_cmd_char}', tx1_addr={tx1_addr}, tx1_cmd='{tx1_cmd_char}'")
        
        try:
            hex_cmd_0 = self.reverse_bits_8bit(self.command_dict[tx0_cmd_char])
            hex_cmd_1 = self.reverse_bits_8bit(self.command_dict[tx1_cmd_char])
            tx0_addr = self.reverse_bits_8bit(tx0_addr)
            tx1_addr = self.reverse_bits_8bit(tx1_addr)
        except KeyError as e:
            print(f"[ERROR] KeyError: '{e.args[0]}' not found in command_dict")
            raise

        print(f"[DEBUG] Command chars mapped to hex: '{tx0_cmd_char}'={hex(hex_cmd_0)}, '{tx1_cmd_char}'={hex(hex_cmd_1)}")

        self.tx0.write(0x00, tx0_addr & 0xFF)
        self.tx0.write(0x08, hex_cmd_0 & 0xFF)
        print(f"[DEBUG] TX0 assigned: addr={hex(tx0_addr & 0xFF)}, cmd={hex(hex_cmd_0 & 0xFF)}")

        self.tx1.write(0x00, tx1_addr & 0xFF)
        self.tx1.write(0x08, hex_cmd_1 & 0xFF)
        print(f"[DEBUG] TX1 assigned: addr={hex(tx1_addr & 0xFF)}, cmd={hex(hex_cmd_1 & 0xFF)}")
    def send_command_full_reversed(self, tx0_addr, tx1_addr, cmd0_str, cmd1_str):
        """Send sequences of commands to both transmitters"""
        print(f"[DEBUG] send_command_full: tx0_addr={tx0_addr}, tx1_addr={tx1_addr}, cmd0='{cmd0_str}', cmd1='{cmd1_str}'")
        
        # Check if strings are same length
        if len(cmd0_str) != len(cmd1_str):
            print(f"[WARNING] Command strings have different lengths: {len(cmd0_str)} vs {len(cmd1_str)}")
        
        # Get the minimum length to iterate safely
        min_len = min(len(cmd0_str), len(cmd1_str))
        
        for idx in range(min_len):
            cmd0_char = cmd0_str[idx]
            cmd1_char = cmd1_str[idx]
            
            print(f"[DEBUG] Loop {idx}: command chars: tx0='{cmd0_char}', tx1='{cmd1_char}'")
            
            # Clear before each iteration
            self._clear_tx(self.tx0)
            self._clear_tx(self.tx1)
            self._btn_low()
            sleep(self.BTN_PULSE_WIDTH)
            
            # Assign values
            self.assign_values_reversed(tx0_addr, cmd0_char, tx1_addr, cmd1_char)
            
            # Pulse button
            print(f"[DEBUG] Pulsing button for commands '{cmd0_char}' and '{cmd1_char}'...")
            self._btn_high()
            sleep(self.BTN_PULSE_WIDTH)
            self._btn_low()
            
            # Hold for NEC frame
            sleep(self.NEC_HOLD_TIME)
        
        print("[DEBUG] send_command_full finished.")



# Example usage
if __name__ == "__main__":
    ir = NEC_IR_Controller(
        "/home/xilinx/jupyter_notebooks/xilinx/overlays/own/design_1_wrapper.bit",
        tx0_base=0x41210000,
        tx1_base=0x41220000,
        btn_base=0x41200000
    )

    try:
        start = datetime.now()

        # Example usage
        tx0_addr = 0x06
        cmd0_str = "-0005+"
        tx1_addr = 0x01
        cmd1_str = "-0006+"

        ir.send_command_full_reversed(tx0_addr, tx1_addr, cmd0_str, cmd1_str)

        print(f"Elapsed: {datetime.now() - start}")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        ir.cleanup()
        print("Cleanup complete")

[DEBUG] Loading overlay from: /home/xilinx/jupyter_notebooks/xilinx/overlays/own/design_1_wrapper.bit
[DEBUG] Overlay loaded and downloaded.
[DEBUG] Initializing MMIO handles...
[DEBUG] MMIO handles initialized: tx0_base=0x41210000, tx1_base=0x41220000, btn_base=0x41200000
[DEBUG] Configuring GPIO directions for transmitters and button...
[DEBUG] GPIO directions configured.
[DEBUG] Clearing transmitter and button states at init...
[DEBUG] Clearing transmitter lines
[DEBUG] Clearing transmitter lines
[DEBUG] Button LOW.
[DEBUG] Initialization complete.
[DEBUG] send_command_full: tx0_addr=6, tx1_addr=1, cmd0='-0005+', cmd1='-0006+'
[DEBUG] Loop 0: command chars: tx0='-', tx1='-'
[DEBUG] Clearing transmitter lines
[DEBUG] Clearing transmitter lines
[DEBUG] Button LOW.
[DEBUG] Assigning values: tx0_addr=6, tx0_cmd='-', tx1_addr=1, tx1_cmd='-'
[DEBUG] Command chars mapped to hex: '-'=0xe0, '-'=0xe0
[DEBUG] TX0 assigned: addr=0x60, cmd=0xe0
[DEBUG] TX1 assigned: addr=0x80, cmd=0xe0
[DEBUG] P